<a href="https://colab.research.google.com/github/SaahilShaikh17/AAIT-Retirement/blob/main/AAIT_Retirement_income.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TEAM AAIT-Retirement Savings Estimator
Team members:
Saahil Shaikh
Sachit Desai
Sanjana Sharma
Sehaj Saluja

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/SaahilShaikh17/AAIT-Retirement/main/nedgroup_training_data.csv")
df_validation=pd.read_csv("https://raw.githubusercontent.com/SaahilShaikh17/AAIT-Retirement/main/nedgroup_validation_data.csv")
df_test=pd.read_csv("https://raw.githubusercontent.com/SaahilShaikh17/AAIT-Retirement/main/nedgroup_testing_data.csv")

In [16]:
pd.set_option('display.max_columns', None)


In [17]:
df.head()


,GENDER,RETIREMENT_AGE,RETIREMENT_FUND_VALUE,DEPT_VALUE,CURRENT_NET_MONTHLY_INCOME,SPARE_CASH_VALUE,FINANCIALLY_SUPPORT_PARTNER,FINANCIALLY_SUPPORT_CHILDREN,YEARS_SUPPORTING_CHILD,CHILD_MONTHLY_SUPPORTING_VALUE,YEARS_SUPPORTING_SOMEONE_ELSE,OTHER_MONTHLY_SUPPORTING_VALUE,HAS_EMERGENCY_SAVINGS,CRITICAL_ILLNESS,ONGOING_COACHING_FEE,CONFIDENCE_LEVEL,INITIAL_PLANNER_FEE_INCL_VAT_UT,INITIAL_PLANNER_FEE_INCL_VAT_LA_AND_LAP,ONGOING_PLANNER_FEE_INCL_VAT_UT,ONGOING_PLANNER_FEE_INCL_VAT_LA_AND_LAP,TARGET_MONTHLY_INCOME,SPOUSE_GENDER,SPOUSE_RETIREMENT_AGE,SPOUSE_DATE_OF_BIRTH,PERCENTAGE_SUCCESS,SA_EQUITY_UNIT_TRUST,SA_BOND_UNIT_TRUST,SA_CASH_UNIT_TRUST,INTERNATIONAL_EQUITY_UNIT_TRUST,INTERNATIONAL_BOND_UNIT_TRUST,INTERNATIONAL_CASH_UNIT_TRUST,SA_EQUITY_LAP,SA_BOND_LAP,SA_CASH_LAP,INTERNATIONAL_EQUITY_LAP,INTERNATIONAL_BOND_LAP,INTERNATIONAL_CASH_LAP,LAP_EAC_PA_INCL_VAT,LA_EAC_PA_INCL_VAT,UNIT_TRUST_EAC_PA_INCL_VAT
0,Female,74,9019817,108988,207,248324,No,No,0,0,4,4412,Yes,Yes,1.086355,65,1.126188,3.994553,0.814560,1.070482,51996,NaN,NaN,NaN,81.984898,0.0,0.0,1.0,90.00,0.00,9.00,36.75,19.75,16.0,18.85,6.9,1.75,1.220000,1.220000,1.220000
1,Female,71,5194248,51928,2013,164644,Yes,Yes,4,4900,2,3382,No,Yes,1.298466,75,3.193294,2.696367,0.615908,2.027214,58303,Male,73.0,1949-01-14,17.274052,0.0,0.0,1.0,59.40,29.70,9.90,24.60,34.40,16.0,17.60,4.4,3.00,1.220000,1.220000,1.220000
2,Female,70,6796815,41684,3047,240103,No,No,0,0,3,2753,Yes,Yes,2.245625,65,0.911502,4.271558,1.872700,0.445151,49510,NaN,NaN,NaN,41.897959,0.0,0.0,1.0,71.28,22.77,4.95,49.00,15.00,6.0,23.00,6.0,1.00,2.945625,2.945625,2.785625
3,Female,78,7238811,147184,1725,20071,No,No,0,0,3,2975,No,Yes,1.396051,75,0.087226,2.354137,0.101542,0.976643,52031,NaN,NaN,NaN,84.565598,0.0,0.0,100.0,0.00,0.00,0.00,49.00,15.00,6.0,23.00,6.0,1.00,0.336955,0.336955,0.350264
4,Male,87,6173619,229014,2261,82670,Yes,Yes,3,294,2,771,No,Yes,0.802640,75,1.387036,2.537295,1.842370,1.320476,57515,Female,61.0,1961-01-20,92.297376,24.5,24.5,26.0,14.70,7.80,2.50,0.00,0.00,1.0,98.00,0.0,1.00,1.460000,1.460000,1.460000


In [15]:
df.shape

(16083, 40)

In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)
df_validation.drop('Unnamed: 0',axis=1,inplace=True)
df_test.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
df.head()

,GENDER,RETIREMENT_AGE,RETIREMENT_FUND_VALUE,DEPT_VALUE,CURRENT_NET_MONTHLY_INCOME,SPARE_CASH_VALUE,FINANCIALLY_SUPPORT_PARTNER,FINANCIALLY_SUPPORT_CHILDREN,YEARS_SUPPORTING_CHILD,CHILD_MONTHLY_SUPPORTING_VALUE,...,INTERNATIONAL_CASH_UNIT_TRUST,SA_EQUITY_LAP,SA_BOND_LAP,SA_CASH_LAP,INTERNATIONAL_EQUITY_LAP,INTERNATIONAL_BOND_LAP,INTERNATIONAL_CASH_LAP,LAP_EAC_PA_INCL_VAT,LA_EAC_PA_INCL_VAT,UNIT_TRUST_EAC_PA_INCL_VAT
0,Female,74,9019817,108988,207,248324,No,No,0,0,...,9.00,36.75,19.75,16.0,18.85,6.9,1.75,1.220000,1.220000,1.220000
1,Female,71,5194248,51928,2013,164644,Yes,Yes,4,4900,...,9.90,24.60,34.40,16.0,17.60,4.4,3.00,1.220000,1.220000,1.220000
2,Female,70,6796815,41684,3047,240103,No,No,0,0,...,4.95,49.00,15.00,6.0,23.00,6.0,1.00,2.945625,2.945625,2.785625
3,Female,78,7238811,147184,1725,20071,No,No,0,0,...,0.00,49.00,15.00,6.0,23.00,6.0,1.00,0.336955,0.336955,0.350264
4,Male,87,6173619,229014,2261,82670,Yes,Yes,3,294,...,2.50,0.00,0.00,1.0,98.00,0.0,1.00,1.460000,1.460000,1.460000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23944 entries, 0 to 23943
Data columns (total 40 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   GENDER                                   23944 non-null  object 
 1   RETIREMENT_AGE                           23944 non-null  int64  
 2   RETIREMENT_FUND_VALUE                    23944 non-null  int64  
 3   DEPT_VALUE                               23944 non-null  int64  
 4   CURRENT_NET_MONTHLY_INCOME               23944 non-null  int64  
 5   SPARE_CASH_VALUE                         23944 non-null  int64  
 6   FINANCIALLY_SUPPORT_PARTNER              23944 non-null  object 
 7   FINANCIALLY_SUPPORT_CHILDREN             23944 non-null  object 
 8   YEARS_SUPPORTING_CHILD                   23944 non-null  int64  
 9   CHILD_MONTHLY_SUPPORTING_VALUE           23944 non-null  int64  
 10  YEARS_SUPPORTING_SOMEONE_ELSE            23944

In [7]:
df.describe()

,RETIREMENT_AGE,RETIREMENT_FUND_VALUE,DEPT_VALUE,CURRENT_NET_MONTHLY_INCOME,SPARE_CASH_VALUE,YEARS_SUPPORTING_CHILD,CHILD_MONTHLY_SUPPORTING_VALUE,YEARS_SUPPORTING_SOMEONE_ELSE,OTHER_MONTHLY_SUPPORTING_VALUE,ONGOING_COACHING_FEE,...,INTERNATIONAL_CASH_UNIT_TRUST,SA_EQUITY_LAP,SA_BOND_LAP,SA_CASH_LAP,INTERNATIONAL_EQUITY_LAP,INTERNATIONAL_BOND_LAP,INTERNATIONAL_CASH_LAP,LAP_EAC_PA_INCL_VAT,LA_EAC_PA_INCL_VAT,UNIT_TRUST_EAC_PA_INCL_VAT
count,23944.000000,2.394400e+04,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,...,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000,23944.000000
mean,74.733503,7.374449e+06,123220.997954,3797.233086,124126.916346,1.265244,1252.694788,2.025601,2008.880471,1.151578,...,2.652024,22.009206,16.874416,20.445863,31.109067,6.910361,2.651087,1.795251,1.795251,1.740949
std,10.352636,3.868508e+06,72175.658095,2166.978695,71892.254230,1.478019,1601.701228,1.414410,1628.818355,0.663169,...,2.075729,18.575175,13.129172,21.716695,22.593260,5.695212,2.065113,0.990233,0.990233,0.974344
min,60.000000,5.001570e+05,32.000000,0.000000,21.000000,0.000000,0.000000,0.000000,0.000000,0.000251,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,66.000000,4.106004e+06,60446.000000,1943.750000,62286.750000,0.000000,0.000000,1.000000,362.000000,0.580646,...,1.500000,3.000000,11.510000,6.000000,18.240000,4.000000,1.500000,0.935803,0.935803,0.912500
50%,73.000000,6.941902e+06,123097.000000,3789.000000,122169.000000,1.000000,66.000000,2.000000,1875.000000,1.154529,...,2.480000,22.845000,15.000000,16.000000,25.500000,6.600000,2.480000,1.730972,1.730972,1.675423
75%,82.000000,1.052333e+07,185062.000000,5693.000000,186600.000000,3.000000,2487.250000,3.000000,3436.000000,1.728732,...,3.185000,31.400000,20.835000,26.000000,41.456000,7.907000,3.185000,2.569211,2.569211,2.499012
max,100.000000,1.499862e+07,249988.000000,7499.000000,249956.000000,4.000000,4998.000000,4.000000,4999.000000,2.299975,...,9.900000,78.900000,63.000000,100.000000,98.000000,29.700000,9.900000,4.613422,4.613422,4.588735


In [8]:
df.isnull().sum()

GENDER                                         0
RETIREMENT_AGE                                 0
RETIREMENT_FUND_VALUE                          0
DEPT_VALUE                                     0
CURRENT_NET_MONTHLY_INCOME                     0
SPARE_CASH_VALUE                               0
FINANCIALLY_SUPPORT_PARTNER                    0
FINANCIALLY_SUPPORT_CHILDREN                   0
YEARS_SUPPORTING_CHILD                         0
CHILD_MONTHLY_SUPPORTING_VALUE                 0
YEARS_SUPPORTING_SOMEONE_ELSE                  0
OTHER_MONTHLY_SUPPORTING_VALUE                 0
HAS_EMERGENCY_SAVINGS                          0
CRITICAL_ILLNESS                               0
ONGOING_COACHING_FEE                           0
CONFIDENCE_LEVEL                               0
INITIAL_PLANNER_FEE_INCL_VAT_UT                0
INITIAL_PLANNER_FEE_INCL_VAT_LA_AND_LAP        0
ONGOING_PLANNER_FEE_INCL_VAT_UT                0
ONGOING_PLANNER_FEE_INCL_VAT_LA_AND_LAP        0
TARGET_MONTHLY_INCOM

In [9]:
df.nunique()

GENDER                                         2
RETIREMENT_AGE                                41
RETIREMENT_FUND_VALUE                      15196
DEPT_VALUE                                 14752
CURRENT_NET_MONTHLY_INCOME                  6478
SPARE_CASH_VALUE                           14779
FINANCIALLY_SUPPORT_PARTNER                    2
FINANCIALLY_SUPPORT_CHILDREN                   2
YEARS_SUPPORTING_CHILD                         5
CHILD_MONTHLY_SUPPORTING_VALUE              3924
YEARS_SUPPORTING_SOMEONE_ELSE                  5
OTHER_MONTHLY_SUPPORTING_VALUE              4578
HAS_EMERGENCY_SAVINGS                          2
CRITICAL_ILLNESS                               2
ONGOING_COACHING_FEE                       15233
CONFIDENCE_LEVEL                               2
INITIAL_PLANNER_FEE_INCL_VAT_UT            15232
INITIAL_PLANNER_FEE_INCL_VAT_LA_AND_LAP    15248
ONGOING_PLANNER_FEE_INCL_VAT_UT            15226
ONGOING_PLANNER_FEE_INCL_VAT_LA_AND_LAP    15224
TARGET_MONTHLY_INCOM

Gender, Financially Support Partner, financially support children, years_supporting_child, Years_supporting_Someone_else, Has_emergency_savings, Confidence_Level, spouse_Gender

So we have 8 categorical variables


## Data Issues

In [10]:
#Check if there are any duplicated values in our dataset
print(df.duplicated().sum())
print(df_validation.duplicated().sum())
print(df_test.duplicated().sum())

7861
2502
0


In [11]:
df.drop_duplicates(inplace=True)
df_validation.drop_duplicates(inplace=True)

In [12]:

print(df.duplicated().sum())
print(df_validation.duplicated().sum())
print(df_test.duplicated().sum())

0
0
0


## EDA


In [12]:
#test push
